In [1]:
!pip install tensorflow

In [1]:
# Import Library
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from google.colab import drive

In [2]:
# Menghubungkan Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path = '/content/drive/MyDrive/dtt/train'
test_path = '/content/drive/MyDrive/dtt/test'

In [4]:
# Menggunakan ImageDataGenerator untuk augmentasi dan normalisasi
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Memuat data training
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),  # Ukuran gambar
    color_mode='grayscale',  # Menggunakan grayscale
    class_mode='sparse',  # Gunakan 'categorical' jika Anda memiliki lebih dari 2 kelas
    batch_size=32,
    seed=42
)

# Memuat data testing
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    color_mode='grayscale',
    class_mode='sparse',
    batch_size=32,
    seed=42
)

Found 136 images belonging to 34 classes.
Found 68 images belonging to 34 classes.


In [7]:
# Membangun model CNN
model = models.Sequential()

# Layer konvolusi pertama
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 1)))
model.add(layers.MaxPooling2D((2, 2)))

# Layer konvolusi kedua
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Layer konvolusi ketiga
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten dan fully connected layer
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(train_generator.num_classes, activation='softmax'))  # Jumlah kelas

# Kompilasi model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=20
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 57s 10s/step - accuracy: 0.0471 - loss: 3.5986 - val_accuracy: 0.0312 - val_loss: 3.5420
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.0000e+00 - loss: 3.5373 - val_accuracy: 0.0000e+00 - val_loss: 3.5577
Epoch 3/20


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0781 - loss: 3.5038 - val_accuracy: 0.0469 - val_loss: 3.5448
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 854ms/step - accuracy: 0.0625 - loss: 3.5249 - val_accuracy: 0.0000e+00 - val_loss: 3.6839
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.0677 - loss: 3.4723 - val_accuracy: 0.0469 - val_loss: 3.5841
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - loss: 3.4412 - val_accuracy: 0.0000e+00 - val_loss: 3.4551
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.0776 - loss: 3.3500 - val_accuracy: 0.0469 - val_loss: 3.7571
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.0312 - loss: 3.3431 - val_accuracy: 0.0000e+00 - val_loss: 3.5168
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.0454 - loss: 3.2919 - val_accuracy: 0.0469 - val_loss: 3.7782
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0312 - loss: 3.3606 - val_accuracy: 0.0000e+00 - val_loss: 2

In [9]:
# Evaluasi model
loss, accuracy = model.evaluate(test_generator)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

# Menyimpan model
model.save('/content/drive/MyDrive/signature_detection_model.h5')  # Simpan model ke Google Drive

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 246ms/step - accuracy: 0.0000e+00 - loss: 4.2454


Validation accuracy: 0.00%


In [10]:
# Fungsi untuk mempersiapkan gambar untuk prediksi
def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150), color_mode='grayscale')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Menambahkan dimensi batch
    img_array /= 255.0  # Normalisasi
    return img_array

In [11]:
# Fungsi untuk memprediksi tanda tangan
def predict_signature(image_path):
    img_array = prepare_image(image_path)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)

    class_labels = list(train_generator.class_indices.keys())  # Mendapatkan label kelas dari generator

    print(f"Prediction for image: {image_path}")
    if predicted_class < len(class_labels):  # Memastikan indeks kelas valid
        predicted_label = class_labels[predicted_class]
        confidence = predictions[0][predicted_class]
        print(f"Predicted class: {predicted_label} (confidence: {confidence:.2f})")
    else:
        print("Error: Predicted class index out of bounds.")

In [12]:
# Contoh penggunaan untuk memprediksi gambar baru
image_path_to_predict = '/content/drive/MyDrive/dtt/ttddr.jpeg'  # Ganti dengan path gambar yang ingin diprediksi
predict_signature(image_path_to_predict)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Prediction for image: /content/drive/MyDrive/dtt/ttddr.jpeg
Predicted class: ainun putri (confidence: 0.05)
